In [1]:
from model import *
m = Model()

In [2]:
def chunk():
    loss = None
    for i in range(50):
        for j in range(100):
            loss = m.train_batch(100)
        print('{}/ loss = {}'.format(i, loss))

In [3]:
chunk()

0/ loss = 0.6766029596328735
1/ loss = 0.6760709881782532
2/ loss = 0.6465603709220886
3/ loss = 0.6405590772628784
4/ loss = 0.5908908247947693
5/ loss = 0.5570366382598877
6/ loss = 0.5018836855888367
7/ loss = 0.4004218280315399
8/ loss = 0.34859463572502136
9/ loss = 0.3764270842075348
10/ loss = 0.2503512501716614
11/ loss = 0.2524663805961609


KeyboardInterrupt: 

In [ ]:
m.score()

In [ ]:
m.sample()